In [1]:
import os

In [2]:
%pwd

'c:\\Users\\670258675\\Music\\New Folder\\Projects\\NLP---Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\670258675\\Music\\New Folder\\Projects\\NLP---Text-Summarizer'

In [5]:
## Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # create root folder
        create_directories([self.config.artifacts_root])
    
    # we have prepared the return type above (DataIngestionConfig)
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [8]:
## For Components
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

c:\Users\670258675\Anaconda3\envs\text\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-24 11:45:52,155: INFO:config:PyTorch version 2.2.1 available.]


In [9]:
import os
from datasets import load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        # Load dataset from disk
        dataset_samsum = load_from_disk(self.config.data_path)

        # Reduce the size of train, test, and validation datasets
        train_data = dataset_samsum['train'].select(range(500))
        test_data = dataset_samsum['test'].select(range(250))
        valid_data = dataset_samsum['validation'].select(range(250))

        # Convert examples to features
        train_dataset = train_data.map(self.convert_examples_to_features, batched=True)
        test_dataset = test_data.map(self.convert_examples_to_features, batched=True)
        valid_dataset = valid_data.map(self.convert_examples_to_features, batched=True)

        # Create directories if they don't exist
        output_dir = os.path.join(self.config.root_dir, "samsum_dataset")
        os.makedirs(output_dir, exist_ok=True)

        # Save datasets to disk
        train_dataset.save_to_disk(os.path.join(output_dir, "train"))
        test_dataset.save_to_disk(os.path.join(output_dir, "test"))
        valid_dataset.save_to_disk(os.path.join(output_dir, "validation"))


In [10]:
# pipeline
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-03-24 11:45:53,447: INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-03-24 11:45:53,463: INFO:common:yaml file: params.yaml loaded successfully]
[2024-03-24 11:45:53,470: INFO:common:Creating directory at: artifacts]
[2024-03-24 11:45:53,470: INFO:common:Creating directory at: artifacts/data_transformation]


Map:   0%|          | 0/500 [00:00<?, ? examples/s]c:\Users\670258675\Anaconda3\envs\text\lib\site-packages\transformers\tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 250/250 [00:00<00:00, 12433.31 examples/s]
